# Homework 2

For this assignment, you will be developing an artificial neural network to classify data given in the __[Dry Beans Data Set](https://archive.ics.uci.edu/ml/datasets/Dry+Bean+Dataset#)__. This data set was obtained as a part of a research study by Selcuk University, Turkey, in which a computer vision system was developed to distinguish seven different registered varieties of dry beans with similar features. More details on the study can be found in the following __[research paper](https://www.sciencedirect.com/science/article/pii/S0168169919311573)__. <br>
**Make sure to use the lecture notebook on an introduction to keras and cross validation located [here](https://colab.research.google.com/drive/1ksEGL7SJ_wutCIyPYx7Loe5EPdOij6dJ?usp=sharing) and [here](https://colab.research.google.com/drive/1C9Mwf1J2ril1Q4l6n2BjQMb8YaFySG5_?usp=sharing)**.

## About the Data Set
Seven different types of dry beans were used in a study in Selcuk University, Turkey, taking into account the features such as form, shape, type, and structure by the market situation. A computer vision system was developed to distinguish seven different registered varieties of dry beans with similar features in order to obtain uniform seed classification. For the classification model, images of 13611 grains of 7 different registered dry beans were taken with a high-resolution camera. Bean images obtained by computer vision system were subjected to segmentation and feature extraction stages, and a total of 16 features - 12 dimensions and 4 shape forms - were obtained from the grains.

Number of Instances (records in the data set): __13611__

Number of Attributes (fields within each record, including the class): __17__

### Data Set Attribute Information:

1. __Area (A)__ : The area of a bean zone and the number of pixels within its boundaries.
2. __Perimeter (P)__ : Bean circumference is defined as the length of its border.
3. __Major axis length (L)__ : The distance between the ends of the longest line that can be drawn from a bean.
4. __Minor axis length (l)__ : The longest line that can be drawn from the bean while standing perpendicular to the main axis.
5. __Aspect ratio (K)__ : Defines the relationship between L and l.
6. __Eccentricity (Ec)__ : Eccentricity of the ellipse having the same moments as the region.
7. __Convex area (C)__ : Number of pixels in the smallest convex polygon that can contain the area of a bean seed.
8. __Equivalent diameter (Ed)__ : The diameter of a circle having the same area as a bean seed area.
9. __Extent (Ex)__ : The ratio of the pixels in the bounding box to the bean area.
10. __Solidity (S)__ : Also known as convexity. The ratio of the pixels in the convex shell to those found in beans.
11. __Roundness (R)__ : Calculated with the following formula: (4piA)/(P^2)
12. __Compactness (CO)__ : Measures the roundness of an object: Ed/L
13. __ShapeFactor1 (SF1)__
14. __ShapeFactor2 (SF2)__
15. __ShapeFactor3 (SF3)__
16. __ShapeFactor4 (SF4)__

17. __Classes : *Seker, Barbunya, Bombay, Cali, Dermosan, Horoz, Sira*__

### Libraries that can be used :
- NumPy, SciPy, Pandas, Sci-Kit Learn, TensorFlow, Keras
- Any other library used during the lectures and discussion sessions.

### Other Notes
- Don't worry about not being able to achieve high accuracy, it is neither the goal nor the grading standard of this assignment.
- Discussion and Lecture materials should be helpful for doing the assignments.
- The homework submission should be a .ipynb file.



## Exercise 1 : Building a Feed-Forward Neural Network(50 points)

### Exercise 1.1 : Data Preprocessing (10 points)

- As the classes are categorical, use one-hot encoding to represent the set of classes. You will find this useful when developing the output layer of the neural network.
- Normalize each field of the input data using the min-max normalization technique.



In [1]:
#!pip install tensorflow
#!pip install keras-tuner

In [2]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn import metrics
from sklearn.model_selection import cross_val_score
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import confusion_matrix
import keras_tuner as kt
import keras

In [3]:
df = pd.read_csv('Dry_Beans_Dataset.csv')
orig_df = df.copy()
print(df.shape)
df.head(5)

(13611, 17)


,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,44830,814.955,320.731947,178.405838,1.797766,0.831018,45297,238.912806,0.658877,0.989690,0.848226,0.744899,0.007154,0.001359,0.554874,0.997534,SIRA
1,33476,691.826,258.837971,165.220760,1.566619,0.769773,33907,206.453305,0.721155,0.987289,0.878921,0.797616,0.007732,0.001930,0.636191,0.996669,DERMASON
2,27057,606.138,227.460904,151.860320,1.497830,0.744491,27358,185.607226,0.801831,0.988998,0.925436,0.815996,0.008407,0.002299,0.665850,0.997330,DERMASON
3,49483,844.283,326.602913,194.689529,1.677558,0.802907,50289,251.005403,0.680179,0.983973,0.872348,0.768534,0.006600,0.001420,0.590644,0.990840,SIRA
4,22461,544.584,192.801303,148.541136,1.297966,0.637517,22699,169.110122,0.774731,0.989515,0.951720,0.877121,0.008584,0.003134,0.769342,0.998579,DERMASON


In [4]:
y = df.Class
y = pd.get_dummies(y,dtype=float)
X = df.drop(['Class'], axis = 1)
sclr = MinMaxScaler()
X = pd.DataFrame(sclr.fit_transform(X), columns = X.columns)
X.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4
0,0.104229,0.198694,0.246967,0.165518,0.549934,0.883887,0.101465,0.190304,0.333127,0.933884,0.715687,0.300877,0.570351,0.256253,0.256074,0.957751
1,0.055748,0.114396,0.135499,0.126473,0.385468,0.795443,0.054511,0.110772,0.533453,0.902047,0.776947,0.452920,0.645632,0.440609,0.400144,0.941143
2,0.028340,0.055731,0.078990,0.086908,0.336523,0.758933,0.027513,0.059695,0.792964,0.924703,0.869780,0.505931,0.733560,0.559513,0.452690,0.953829
3,0.124097,0.218773,0.257541,0.213740,0.464403,0.843291,0.122044,0.219934,0.401647,0.858085,0.763829,0.369044,0.498138,0.276116,0.319448,0.829136
4,0.008715,0.013589,0.016569,0.077079,0.194315,0.604452,0.008307,0.019274,0.705791,0.931560,0.922235,0.682223,0.756639,0.828764,0.636047,0.977839


In [5]:
y.head(5)

,BARBUNYA,BOMBAY,CALI,DERMASON,HOROZ,SEKER,SIRA
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0


### Exercise 1.2 : Training and Testing the Neural Network (40 points)

Design a 4-layer (4 hidden layers and this does not include the output layer) artificial deep neural network, specifically a feed-forward multi-layer perceptron (using the sigmoid activation function), to classify the type of 'Dry Bean' given the other attributes in the data set, similar to the one mentioned in the paper above. Please note that this is a multi-class classification problem so select the right number of nodes accordingly for the output layer.

For training and testing the model, split the data into training and testing set by __90:10__ and use the training set for training the model and the test set to evaluate the model performance.

Consider the following hyperparameters while developing your model :

- Model type: Keras Sequential
- Number and type of layers: 4 and Dense
- Number of nodes in each hidden layer: 12
- Learning rate should be 0.3
- Number of epochs should be 100
- The sigmoid function should be used as the activation function in each layer
- Stochastic Gradient Descent should be used to minimize the error rate

__Requirements once the model has been trained :__

- A confusion matrix for all classes, specifying the true positive, true negative, false positive, and false negative cases for each category in the class
- The accuracy and mean squared error (MSE) of the model
- The precision and recall for each label in the class

__Notes :__

- Splitting of the dataset should be done __after__ the data preprocessing step.
- The mean squared error (MSE) values obtained __should be positive__.


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [7]:
#model
model = Sequential(
    [
    Dense(12, input_dim=X.shape[1], activation='sigmoid', name="input"),
    Dense(12, activation='sigmoid', name="layer1"),
    Dense(12, activation='sigmoid', name="layer2"),
    Dense(12, activation='sigmoid', name="layer3"),
    Dense(orig_df.Class.unique().size, activation='softmax', name="output")
    ]
)

sgd = SGD(learning_rate=0.3)


#compile
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy','mse']) #use mse or categorical_crossentropy for loss?

In [8]:
model.fit(X_train, y_train, epochs=100, validation_data = (X_test, y_test),verbose=0)

In [9]:
y_pred = model.predict(X_test,verbose=0) #predicting

y_pred_flat = np.argmax(y_pred,axis=1) #flattening into 1-d array, with value representing class it was classfied as
y_test_flat = np.argmax(y_test.values,axis=1) #formatting the test y data to be the same

conf_m = tf.math.confusion_matrix(y_test_flat,y_pred_flat) #creating confusion matrix
conf_m = pd.DataFrame(conf_m, columns=y.columns, index=y.columns)
full_conf_m=pd.concat({'Observations': conf_m}, names=['Predictions']) #clarifying confusion matrix

#helper func for the multi-class accuraccy and recall
def get_metrics(df, model):
    metrics = model.get_metrics_result()
    mse = metrics['mse'].numpy()
    acc = metrics['accuracy'].numpy()
    print('Model MSE:', mse)
    print('Model Accuracy:', acc)
    for cl in df.index:
        tp = df.loc[cl][cl]
        fp = df[cl].sum() - tp
        fn = df.loc[cl].sum()-tp
        print(cl,'recall:',round((tp/(tp+fn)),5),'precision:',round((tp/(tp+fp)),5))

    return

In [10]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Test mse:", score[2])
print("Test accuracy:", score[1])
get_metrics(conf_m, model)
print('Confusion Matrix:')
full_conf_m

Test mse: 0.019593754783272743
Test accuracy: 0.9060205817222595
Model MSE: 0.019593755
Model Accuracy: 0.9060206
BARBUNYA recall: 0.92701 precision: 0.92701
BOMBAY recall: 1.0 precision: 1.0
CALI recall: 0.9641 precision: 0.92611
DERMASON recall: 0.84211 precision: 0.93506
HOROZ recall: 0.96133 precision: 0.90155
SEKER recall: 0.905 precision: 0.97312
SIRA recall: 0.87295 precision: 0.78309
Confusion Matrix:


BARBUNYA  BOMBAY  CALI  DERMASON  HOROZ  SEKER  SIRA
Predictions                                                                
Observations BARBUNYA       127       0     7         0      2      1     0
             BOMBAY           0      63     0         0      0      0     0
             CALI             2       0   188         0      4      1     0
             DERMASON         0       0     0       288      3      1    50
             HOROZ            0       0     6         0    174      0     1
             SEKER            6       0     0         5      0    181     8
             SIRA             2       0     2        15     10      2   213

## Exercise 2 : k-fold Cross Validation (20 points)

In order to avoid using biased models, use 10-fold cross validation to generalize the model from Ex1.2 on the given data set. You can choose a n_repeats value of 1-5

__Requirements :__
- Print the accuracy and MSE values during each iteration of the cross validation
- Print the overall average accuracy and MSE value

__Note :__ The mean squared error (MSE) values obtained should be positive.

In [34]:
def buildmodel():
    model = Sequential(
        [
        Dense(12, input_dim=X.shape[1], activation='sigmoid', name="input"),
        Dense(12, activation='sigmoid', name="layer1"),
        Dense(12, activation='sigmoid', name="layer2"),
        Dense(12, activation='sigmoid', name="layer3"),
        Dense(orig_df.Class.unique().size, activation='softmax', name="output")
        ]
    )
    sgd = SGD(learning_rate=0.3)

    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy','mse'])
    return(model)

In [35]:
kFold = RepeatedKFold(n_splits=10, n_repeats=1)

mselist=[]
acclist=[]
for k, (train, test) in enumerate(kFold.split(X, y)):
    testmodel = buildmodel()
    testmodel.fit(X.iloc[train], y.iloc[train],epochs=100,validation_data=(X.iloc[test], y.iloc[test]),verbose=0)
    metrics = testmodel.get_metrics_result()
    mse = metrics['mse'].numpy()
    acc = metrics['accuracy'].numpy()
    mselist.append(mse)
    acclist.append(acc)
    print(
        "[fold {0}] mse: {1:.5f}, acc: {2:.5f}".format(
            k+1, mse, acc)
        )
    i = train
print('overall mean mse:', np.mean(mselist))
print('overall mean accuracy:', np.mean(acclist))

[fold 1] mse: 0.01974, acc: 0.90235
[fold 2] mse: 0.01469, acc: 0.92579
[fold 3] mse: 0.01580, acc: 0.92652
[fold 4] mse: 0.01841, acc: 0.91771
[fold 5] mse: 0.01684, acc: 0.92065
[fold 6] mse: 0.01993, acc: 0.90375
[fold 7] mse: 0.01676, acc: 0.91844
[fold 8] mse: 0.01556, acc: 0.92359
[fold 9] mse: 0.01677, acc: 0.92212
[fold 10] mse: 0.01600, acc: 0.92579
overall mean mse: 0.017050643
overall mean accuracy: 0.9186699


## Exercise 3 : Hyperparameter Tuning (25 points)

Use either grid search or random search methodology to find the optimal number of nodes required in each hidden layer, as well as the optimal learning rate and the different activation functions or optimization approaches, [keras_tuner examples](https://keras.io/guides/keras_tuner/getting_started/) such that the accuracy of the model is maximum for the given data set.

__Requirements :__
- The set of optimal hyperparameters
- The maximum accuracy achieved using this set of optimal hyperparameters

__Note :__ Hyperparameter tuning takes a lot of time to execute. Make sure that you choose the appropriate number of each hyperparameter (preferably 3 of each), and that you allocate enough time to execute your code.

In [31]:
#hp.Float("lr", min_value=1e-4, max_value=0.5, sampling="log")

0.0001

In [36]:
hp = kt.HyperParameters()
def build_model(hp):
    
    node_param = hp.Int("units", min_value=1, max_value=15, step=1)
    activation=hp.Choice("activation", ["sigmoid", "relu","tanh"])
    lr = hp.Choice('lr', [0.1,0.3,0.5])
    loss_param = hp.Choice('loss', ['categorical_crossentropy','mse','categorical_hinge'])
    
    model = Sequential(
    [Dense(node_param, input_dim=X.shape[1], activation=activation, name="input"),
    Dense(node_param, activation=activation, name="layer1"),
    Dense(node_param, activation=activation, name="layer2"),
    Dense(node_param, activation=activation, name="layer3"),
    Dense(orig_df.Class.unique().size, activation='softmax', name="output")]
    )

    sgd = SGD(learning_rate=lr)
    
    model.compile(optimizer=sgd, loss=loss_param, metrics=['accuracy','mse'])
    return model

build_model(kt.HyperParameters())

tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="accuracy",
    executions_per_trial=1,
    max_trials=50,
    overwrite=True,
)
build_model(kt.HyperParameters())

In [37]:
tuner.search(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

Trial 50 Complete [00h 00m 38s]
accuracy: 0.9258714914321899

Best accuracy So Far: 0.9270960688591003
Total elapsed time: 00h 32m 28s


In [38]:
models = tuner.get_best_models(num_models=1)
best_model = models[0]
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (Dense)               (None, 13)                221       
                                                                 
 layer1 (Dense)              (None, 13)                182       
                                                                 
 layer2 (Dense)              (None, 13)                182       
                                                                 
 layer3 (Dense)              (None, 13)                182       
                                                                 
 output (Dense)              (None, 7)                 98        
                                                                 
Total params: 865 (3.38 KB)
Trainable params: 865 (3.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [39]:
tuner.results_summary(1)

Results summary
Results in .\untitled_project
Showing 1 best trials
Objective(name="accuracy", direction="max")

Trial 36 summary
Hyperparameters:
units: 13
activation: tanh
lr: 0.1
loss: categorical_crossentropy
Score: 0.9270960688591003


## Exercise 4 - Collaborative Statement (5 points)

It is mandatory to include a Statement of Collaboration in each submission, that follows the guidelines below.
Include the names of everyone involved in the discussions (especially in-person ones), and what was discussed.
All students are required to follow the academic honesty guidelines posted on the course website. For
programming assignments in particular, I encourage students to organize (perhaps using Piazza) to discuss the
task descriptions, requirements, possible bugs in the support code, and the relevant technical content before they
start working on it. However, you should not discuss the specific solutions, and as a guiding principle, you are
not allowed to take anything written or drawn away from these discussions (no photographs of the blackboard,
written notes, referring to Piazza, etc.). Especially after you have started working on the assignment, try to restrict
the discussion to Piazza as much as possible, so that there is no doubt as to the extent of your collaboration.